In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time

### 0. 자동화를 위해 BASE_URL 설정 후 셀러니움 작동 여부 확인

In [2]:
BASE_URL = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sort=2&photo=0&field=0&pd=3&"
# https://finance.naver.com/sise/sise_market_sum.naver?&page=1
search_span= "ds=1998.05.20&de=1998.12.31&a:all&start="

BASE_URL = BASE_URL + search_span

print(BASE_URL)

https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sort=2&photo=0&field=0&pd=3&ds=1998.05.20&de=1998.12.31&a:all&start=


In [3]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(BASE_URL+"1")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/krc/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.134/chromedriver] found in cache


### 1. 마지막 페이지 번호 찾기

"맨뒤"라는 별도의 서치버튼이 없어서 아래 방식 중에 하나로 찾아야 함
1) 휴먼 서치 (직접 찾기)
2) 임의의 숫자(마지막 페이지)을 입력하고 역순으로 조회해서 처음으로 에러가 안나오면 해당 숫자가 마지막 페이지

In [4]:
# 우선은 직접 찾아봄 (기간: ds=1998.05.20&de=1998.12.31 )
num_last_page = 87

In [5]:
print(num_last_page)
driver.close()

87


### 2. 페이지 번호 변경하면서 데이터 긁어오기

In [6]:
# del [[tables]]

In [17]:
columns = ['date', 'title', 'link']
tables = pd.DataFrame(columns=columns)
tables

,date,title,link


In [18]:
for i in range(0, num_last_page+1):
    try:
        res = requests.get(BASE_URL+f"{i}"+"1")
        time.sleep(1)
        print(res.status_code)
        soup = BeautifulSoup(res.content, 'html.parser', from_encoding='utf-8')

        soup_date = soup.select("span.info")
        soup_info = soup.select("a.news_tit")

        dates = [date.get_text() for date in soup_date]
        titles = [title.get_text() for title in soup_info]
        links = [link.get('href') for link in soup_info]

        table = {'date': dates, 'title': titles, 'link': links}

        columns = ['date', 'title', 'link']
        table = pd.DataFrame(table, columns=columns)
        tables = pd.concat([tables, table], axis=0, sort=False)

    except:
        print("Something went wrong.")

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


### Check results

In [21]:
tables.head(30)

,date,title,link
0,1998.05.20.,주가 사흘째 상승세,https://n.news.naver.com/mnews/article/001/000...
1,1998.05.20.,"아남텔레콤, PDA 데이터 전송서비스 개시",https://n.news.naver.com/mnews/article/001/000...
2,1998.05.20.,"<회전목마> 삼성전자, 사이버 사외보 제작",https://n.news.naver.com/mnews/article/001/000...
3,1998.05.20.,"<주식시황> 사흘째 상승, 3백70선 다가서",https://n.news.naver.com/mnews/article/001/000...
4,1998.05.20.,"-춘계승마- 신창무,싼졸쥬마장마술 우승",https://sports.news.naver.com/news.nhn?oid=001...
5,1998.05.21.,주가 3백70선 회복,https://n.news.naver.com/mnews/article/001/000...
6,1998.05.21.,`반도체업체는 여전히 안정된 직장',https://n.news.naver.com/mnews/article/001/000...
7,1998.05.21.,휴대형 입체영상 모니터 국내 첫 개발,https://n.news.naver.com/mnews/article/001/000...
8,1998.05.21.,"<체육단신> 삼성전자, 스포츠전문 웹진 개설",https://sports.news.naver.com/news.nhn?oid=001...
9,1998.05.21.,임금삭감으로 직장인들 보릿고개 고통,https://n.news.naver.com/mnews/article/001/000...


In [22]:
tables.tail(30)

,date,title,link
6,1998.12.21.,"5대재벌 유상증자 작년 10배, 전체의 64% 독식",https://n.news.naver.com/mnews/article/001/000...
7,1998.12.21.,완전평면TV 시장 경쟁 치열,https://n.news.naver.com/mnews/article/001/000...
8,1998.12.21.,삼성-대우 빅딜 SM5 협상 여전히 난항(종합),https://n.news.naver.com/mnews/article/001/000...
9,1998.12.21.,"주가 40포인트 폭등, 5백60선 회복(종합)",https://n.news.naver.com/mnews/article/001/000...
0,1998.12.21.,올해 수출비중 높은 기업 매출.수익 호전,https://n.news.naver.com/mnews/article/001/000...
1,1998.12.22.,"삼성,전력용반도체 4억5천5백만달러에 매각(종합)",https://n.news.naver.com/mnews/article/001/000...
2,1998.12.22.,"한통프리텔, 벨소리 선택 서비스제공",https://n.news.naver.com/mnews/article/001/000...
3,1998.12.22.,10월이후 외국인 주식투자이익 74억달러,https://n.news.naver.com/mnews/article/001/000...
4,1998.12.22.,"삼성, 퇴출대상 이천전기 일진에 매각",https://n.news.naver.com/mnews/article/001/000...
5,1998.12.22.,"<주식시황> 주가 하락세 반전, 5백40선으로 밀려",https://n.news.naver.com/mnews/article/001/000...


In [23]:
tables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 876 entries, 0 to 5
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    876 non-null    object
 1   title   876 non-null    object
 2   link    876 non-null    object
dtypes: object(3)
memory usage: 27.4+ KB


In [40]:
# 중복된 링크가 없음을 확인 (각각 고유한 기사)
len(tables.link.unique())

876

In [26]:
tables.to_excel("articles_samsung_electronics.xlsx", sheet_name="se_articles", index=False)